# getting number of clusters louvain + gmm

In [ ]:
import scanpy as sc
from sklearn.metrics import silhouette_score,calinski_harabasz_score, davies_bouldin_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

In [ ]:
dataset = ''

In [ ]:

batch_key_dic = {'Immune_Human' : 'batch',
                 'Lung' : 'batch',
                 'Pancreas' : 'tech',
                 'Human_Mouse' : 'batch',
                 'Human_Retina': "Batch",
                 
                }
cell_type_key_dic = {'Immune_Human' : 'final_annotation',
                 'Lung' : 'cell_type',
                 'Pancreas' : 'celltype',
                 'Human_Mouse' : "celltype",
                 "Human_Retina":"Subcluster",
                    }    

dir_path = "/home/krushna/Documents/Data_integration/SCRNA_Datasets/All_h5ad/"
emb_path = "/home/krushna/Documents/Data_integration/Othermethods/ourmethod/self-sup/pipeline_tf2 50/step1/"
batch = batch_key_dic[dataset]
cell_type = cell_type_key_dic[dataset]

In [ ]:
adata = sc.read_h5ad(dir_path+dataset+'.h5ad')
emb_path = emb_path + dataset+'_latent_matrix_c99_step1.csv'
final_embedings=pd.read_csv(emb_path, sep=',',header=None).values
adata.obsm['final_embeddings'] = final_embedings
sc.pp.neighbors(adata, use_rep='final_embeddings')
adata

In [ ]:
number_of_celltypes = len(adata.obs[cell_type].unique())
print('number of cell types ',  number_of_celltypes)

In [ ]:
cluster_res = []
for res in list(np.linspace(0.0001, 1.5, num=20)):
    #find clusters
    sc.tl.leiden(adata, resolution=res) # louvain
    noofclus = len(np.unique(adata.obs['leiden']))
    cluster_res.append([noofclus,res])

In [ ]:
cluster_res

In [ ]:
# find clusters
clusters = []
sil_scores = []
ch_scores = []
bic_scores = []
db_scores = []
for res in list(np.linspace(0.0001, 1.5, num=20)):
    #find clusters
    sc.tl.leiden(adata, resolution=res) # louvain
    noofclus = len(np.unique(adata.obs['leiden']))
    
    if noofclus == 1:
        continue
        
    clusters.append(noofclus)
    # silhouette_score
    sil_score = silhouette_score(adata.obsm['final_embeddings'], adata.obs['leiden'])
    sil_scores.append(sil_score)
    
    #ch score
    ch = calinski_harabasz_score(adata.obsm['final_embeddings'], adata.obs['leiden'])
    ch_scores.append(ch)
    
    #db scores
    db = davies_bouldin_score(adata.obsm['final_embeddings'], adata.obs['leiden'])
    db_scores.append(db)
    
    # bic 
    gmm = GaussianMixture(n_components=noofclus).fit(adata.obsm['final_embeddings'])
    labels = gmm.predict(adata.obsm['final_embeddings'])
    bic = gmm.bic(adata.obsm['final_embeddings'])
    bic_scores.append(bic)

In [ ]:
pd.DataFrame([clusters,ch_scores,sil_scores, bic_scores, db_scores])

In [ ]:
plt.plot(clusters, ch_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('ch_scores')
plt.title('ch_scores '+dataset+' ideal '+ str(number_of_celltypes))
plt.show()

plt.plot(clusters, sil_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('sil_scores')
plt.title('sil_scores '+dataset+' ideal '+ str(number_of_celltypes))
plt.show()

plt.plot(clusters, bic_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('bic_scores')
plt.title('bic_scores '+dataset+' ideal '+ str(number_of_celltypes))
plt.show()

plt.plot(clusters, db_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('db_scores')
plt.title('db_scores '+dataset+' ideal '+ str(number_of_celltypes))
plt.show()

In [ ]:
slopes = []
for i in range(len(clusters)-1):
    x1, x2 = clusters[i], clusters[i+1]
    y1, y2 = bic_scores[i], bic_scores[i+1]
    slope = (y2-y1)/(x2-x1)
    slopes.append(slope)
    
# bic_deriv = np.gradient(bics, clusters)

# plot derivative of BIC curve
plt.plot(clusters[:-1], slopes, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('BIC derivative')
plt.title('Derivative of BIC curve '+ dataset +' ideal ' + str(number_of_celltypes) )
plt.show()
    

In [ ]:
# bic_deriv = np.gradient(bics, clusters)

# # plot derivative of BIC curve
# plt.plot(clusters, bic_deriv, marker='o')
# plt.xlabel('Number of clusters')
# plt.ylabel('BIC derivative')
# plt.title('Derivative of BIC curve')
# plt.show()